In [11]:
import pandas as pd
df = pd.read_csv('D:\\dados\\bar_data.csv')
df.head(5)


,DateTime,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,...,SMA50,SMA100,SMA200,StochasticoK,StochasticoD,RSI,MACD,MACDSignal,MACDHistogram,Gatilho
0,2023-03-20 01:20:00,14019.50,14021.25,14018.50,14021.25,68,0.00,1.00,1.75,2.75,...,14034.93,14038.4150,14033.47625,39.375000,38.431360,39.201736,-4.002780,-4.596095,0.593315,0
1,2023-03-20 01:21:00,14021.75,14025.50,14021.25,14023.75,108,1.75,0.50,2.00,4.25,...,14034.39,14038.2750,14033.41250,55.000000,38.292004,45.257446,-3.554564,-4.387789,0.833225,0
2,2023-03-20 01:22:00,14023.25,14024.50,14019.75,14022.00,62,1.25,2.25,-1.25,4.75,...,14033.85,14038.1100,14033.36250,60.000000,38.708888,42.096598,-3.302491,-4.170729,0.868239,0
3,2023-03-20 01:23:00,14021.00,14021.50,14017.75,14019.25,83,0.50,1.50,-1.75,3.75,...,14033.30,14037.9175,14033.41750,48.118280,38.578665,37.646977,-3.286736,-3.993931,0.707195,0
4,2023-03-20 01:24:00,14019.50,14025.25,14018.25,14023.50,127,1.75,1.25,4.00,7.00,...,14032.87,14037.7550,14033.38250,48.673835,40.052084,46.975146,-2.897905,-3.774725,0.876821,0


In [12]:
data = df.copy()

# Converter a coluna DateTime para datetime
data['DateTime'] = pd.to_datetime(data['DateTime'])

# Definir a coluna Gatilho com valores iniciais iguais a 0
data['Gatilho'] = 0

# Definir os horários de verão e não verão
horario_verao_inicio = pd.Timestamp('10:30:00').time()
horario_verao_fim = pd.Timestamp('11:00:00').time()

horario_nao_verao_inicio = pd.Timestamp('11:30:00').time()
horario_nao_verao_fim = pd.Timestamp('12:00:00').time()

# Função para determinar se a data está no horário de verão dos EUA
def is_dst(date):
    year = date.year
    # Segundo domingo de março
    dst_start = pd.Timestamp(year, 3, 1) + pd.DateOffset(weeks=2, weekday=6)
    # Primeiro domingo de novembro
    dst_end = pd.Timestamp(year, 11, 1) + pd.DateOffset(weekday=6)
    return dst_start <= date < dst_end

# Aplicar a lógica para definir o valor da coluna Gatilho
for idx, row in data.iterrows():
    current_time = row['DateTime'].time()
    if is_dst(row['DateTime']):
        if horario_verao_inicio <= current_time <= horario_verao_fim:
            data.at[idx, 'Gatilho'] = 1
    else:
        if horario_nao_verao_inicio <= current_time <= horario_nao_verao_fim:
            data.at[idx, 'Gatilho'] = 1

# Contar o número de dias úteis (de segunda a sexta-feira) no dataframe
dias_uteis = data['DateTime'].dt.date[data['DateTime'].dt.weekday < 5].nunique()
print(f"Número de dias úteis no dataframe: {dias_uteis}")




Número de dias úteis no dataframe: 379


In [13]:
# quero saber quantas linhas no dataframe têm Gatilho = 1
num_gatilho = data['Gatilho'].sum()
print(f"Número de linhas com Gatilho = 1: {num_gatilho}")


Número de linhas com Gatilho = 1: 11656


In [14]:
data

,DateTime,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,...,SMA50,SMA100,SMA200,StochasticoK,StochasticoD,RSI,MACD,MACDSignal,MACDHistogram,Gatilho
0,2023-03-20 01:20:00,14019.50,14021.25,14018.50,14021.25,68,0.00,1.00,1.75,2.75,...,14034.930,14038.4150,14033.47625,39.375000,38.431360,39.201736,-4.002780,-4.596095,0.593315,0
1,2023-03-20 01:21:00,14021.75,14025.50,14021.25,14023.75,108,1.75,0.50,2.00,4.25,...,14034.390,14038.2750,14033.41250,55.000000,38.292004,45.257446,-3.554564,-4.387789,0.833225,0
2,2023-03-20 01:22:00,14023.25,14024.50,14019.75,14022.00,62,1.25,2.25,-1.25,4.75,...,14033.850,14038.1100,14033.36250,60.000000,38.708888,42.096598,-3.302491,-4.170729,0.868239,0
3,2023-03-20 01:23:00,14021.00,14021.50,14017.75,14019.25,83,0.50,1.50,-1.75,3.75,...,14033.300,14037.9175,14033.41750,48.118280,38.578665,37.646977,-3.286736,-3.993931,0.707195,0
4,2023-03-20 01:24:00,14019.50,14025.25,14018.25,14023.50,127,1.75,1.25,4.00,7.00,...,14032.870,14037.7550,14033.38250,48.673835,40.052084,46.975146,-2.897905,-3.774725,0.876821,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484403,2024-08-30 17:56:00,19831.00,19831.00,19826.75,19829.00,120,0.00,2.25,-2.00,4.25,...,19836.280,19815.3925,19768.55625,24.696623,44.708510,37.472246,-1.450395,-0.913543,-0.536853,0
484404,2024-08-30 17:57:00,19829.00,19832.25,19828.50,19831.25,100,1.00,0.50,2.25,3.75,...,19835.875,19816.1450,19769.14375,46.706192,44.991506,43.903797,-1.439444,-1.018723,-0.420721,0
484405,2024-08-30 17:58:00,19831.25,19832.75,19829.00,19829.75,111,1.50,0.75,-1.50,3.75,...,19835.710,19816.8875,19769.75125,57.575758,46.646137,40.884570,-1.534118,-1.121802,-0.412316,0
484406,2024-08-30 17:59:00,19830.00,19831.00,19826.75,19829.00,83,1.00,2.25,-1.00,4.25,...,19835.530,19817.6300,19770.35000,56.439394,46.795280,39.424692,-1.650638,-1.227569,-0.423069,0


In [6]:
#cria um novo dataframe com apenas as primeiras 16 colunas
df2 = data.iloc[:, 0:16]
df2

,DateTime,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,D_Open,D_High,D_Low,D_Close,D_PavSup,D_PavInf
0,2024-01-02 02:21:00,17756.75,17757.00,17755.75,17756.75,41,0.25,1.00,0.00,1.25,-409,-400,-411,-404,4,2
1,2024-01-02 02:22:00,17757.00,17758.25,17756.75,17758.25,30,0.00,0.25,1.25,1.50,-404,-390,-404,-390,0,0
2,2024-01-02 02:23:00,17758.25,17759.25,17757.75,17758.75,132,0.50,0.50,0.50,1.50,-390,-374,-418,-418,16,0
3,2024-01-02 02:24:00,17758.50,17758.50,17755.50,17755.50,55,0.00,0.00,-3.00,3.00,-418,-418,-437,-437,0,0
4,2024-01-02 02:25:00,17755.75,17757.50,17755.00,17757.50,75,0.00,0.75,1.75,2.50,-437,-437,-457,-450,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223154,2024-08-30 17:56:00,19831.00,19831.00,19826.75,19829.00,120,0.00,2.25,-2.00,4.25,1010,1011,963,996,1,33
223155,2024-08-30 17:57:00,19829.00,19832.25,19828.50,19831.25,100,1.00,0.50,2.25,3.75,996,1034,995,1024,10,1
223156,2024-08-30 17:58:00,19831.25,19832.75,19829.00,19829.75,111,1.50,0.75,-1.50,3.75,1024,1041,1010,1011,17,1
223157,2024-08-30 17:59:00,19830.00,19831.00,19826.75,19829.00,83,1.00,2.25,-1.00,4.25,1011,1016,982,996,5,14


In [13]:
# Encontrar o índice do primeiro valor diferente de zero na coluna D_Open
first_non_zero_index = df2[df2['D_Open'] != 0].index[0]

# Filtrar o dataframe a partir desse índice
df_filtered = df2.loc[first_non_zero_index:]

df_filtered.head()

,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,D_Open,D_High,D_Low,D_Close,D_PavSup,D_PavInf
DateTime,,,,,,,,,,,,,,,
2023-03-20 22:01:00,14026.50,14027.50,14016.50,14018.00,292,1.0,1.50,-8.50,11.00,1,12,-84,-83,11,1
2023-03-20 22:02:00,14017.00,14019.75,14015.25,14017.25,101,2.5,1.75,0.25,4.50,-83,-75,-108,-104,8,4
2023-03-20 22:03:00,14016.50,14018.00,14012.25,14013.75,86,1.5,1.50,-2.75,5.75,-104,-100,-137,-136,4,1
2023-03-20 22:04:00,14013.25,14014.75,14012.00,14014.75,48,0.0,1.25,1.50,2.75,-136,-135,-152,-144,1,8
2023-03-20 22:05:00,14014.50,14017.75,14014.50,14017.75,38,0.0,0.00,3.25,3.25,-144,-116,-144,-116,0,0


In [14]:
df_filtered

,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,D_Open,D_High,D_Low,D_Close,D_PavSup,D_PavInf
DateTime,,,,,,,,,,,,,,,
2023-03-20 22:01:00,14026.50,14027.50,14016.50,14018.00,292,1.00,1.50,-8.50,11.00,1,12,-84,-83,11,1
2023-03-20 22:02:00,14017.00,14019.75,14015.25,14017.25,101,2.50,1.75,0.25,4.50,-83,-75,-108,-104,8,4
2023-03-20 22:03:00,14016.50,14018.00,14012.25,14013.75,86,1.50,1.50,-2.75,5.75,-104,-100,-137,-136,4,1
2023-03-20 22:04:00,14013.25,14014.75,14012.00,14014.75,48,0.00,1.25,1.50,2.75,-136,-135,-152,-144,1,8
2023-03-20 22:05:00,14014.50,14017.75,14014.50,14017.75,38,0.00,0.00,3.25,3.25,-144,-116,-144,-116,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 18:56:00,17759.50,17769.50,17756.25,17769.25,985,0.25,3.25,9.75,13.25,-13045,-12965,-13124,-12970,5,79
2023-12-29 18:57:00,17768.75,17770.25,17766.50,17767.50,606,1.50,1.00,-1.25,3.75,-12970,-12922,-12994,-12968,46,24
2023-12-29 18:58:00,17768.00,17769.00,17763.00,17768.75,677,0.25,5.00,0.75,6.00,-12968,-12935,-13072,-13001,33,71


In [11]:
# converte a coluna DateTime para index date time
df2.set_index('DateTime', inplace=True)
df2

,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,D_Open,D_High,D_Low,D_Close,D_PavSup,D_PavInf
DateTime,,,,,,,,,,,,,,,
2020-01-02 02:21:00,10390.75,10391.00,10390.50,10390.75,33,0.25,0.25,0.00,0.50,0,0,0,0,0,0
2020-01-02 02:22:00,10390.75,10392.00,10390.75,10392.00,31,0.00,0.00,1.25,1.25,0,0,0,0,0,0
2020-01-02 02:23:00,10392.25,10392.25,10391.50,10391.50,28,0.00,0.00,-0.75,0.75,0,0,0,0,0,0
2020-01-02 02:24:00,10392.00,10392.00,10390.75,10392.00,32,0.00,1.25,0.00,1.25,0,0,0,0,0,0
2020-01-02 02:25:00,10391.75,10392.25,10391.75,10392.25,22,0.00,0.00,0.50,0.50,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29 18:56:00,17759.50,17769.50,17756.25,17769.25,985,0.25,3.25,9.75,13.25,-13045,-12965,-13124,-12970,5,79
2023-12-29 18:57:00,17768.75,17770.25,17766.50,17767.50,606,1.50,1.00,-1.25,3.75,-12970,-12922,-12994,-12968,46,24
2023-12-29 18:58:00,17768.00,17769.00,17763.00,17768.75,677,0.25,5.00,0.75,6.00,-12968,-12935,-13072,-13001,33,71


In [12]:

# filtre o horario de 9:30 as 9:58
df2.index = pd.to_datetime(df2.index)
df3 = df2.between_time('10:28', '10:58')

df3.head(40)

,Open,High,Low,Close,Volume,PavioSuperior,PavioInferior,Corpo,Range,D_Open,D_High,D_Low,D_Close,D_PavSup,D_PavInf
DateTime,,,,,,,,,,,,,,,
2020-01-02 10:28:00,10426.75,10427.75,10426.75,10427.50,112,0.25,0.00,0.75,1.00,0,0,0,0,0,0
2020-01-02 10:29:00,10427.50,10427.50,10427.25,10427.50,47,0.00,0.25,0.00,0.25,0,0,0,0,0,0
2020-01-02 10:30:00,10427.25,10427.75,10427.00,10427.50,82,0.25,0.25,0.25,0.75,0,0,0,0,0,0
2020-01-02 10:31:00,10427.25,10428.00,10426.50,10426.75,127,0.75,0.25,-0.50,1.50,0,0,0,0,0,0
2020-01-02 10:32:00,10426.50,10427.50,10426.25,10427.50,84,0.00,0.25,1.00,1.25,0,0,0,0,0,0
2020-01-02 10:33:00,10427.25,10427.25,10426.25,10426.50,79,0.00,0.25,-0.75,1.00,0,0,0,0,0,0
2020-01-02 10:34:00,10426.25,10427.00,10426.00,10426.50,41,0.50,0.25,0.25,1.00,0,0,0,0,0,0
2020-01-02 10:35:00,10426.50,10426.75,10425.75,10426.50,48,0.25,0.75,0.00,1.00,0,0,0,0,0,0
2020-01-02 10:36:00,10426.50,10427.50,10426.00,10427.00,105,0.50,0.50,0.50,1.50,0,0,0,0,0,0
